---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [ ]:
import pandas as pd
import re


def turnToDate(df):
    def myMonth(s):
        s3 = s[:3]
        m = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"].index(s3)
        return str(m + 1)
    if df.re1 != -1:
        y = df.re1[-4:]
        a = df.re1[:-4].rstrip()
        a = a.split(" ")
        if len(a) == 1 :
            return myMonth(a[0]) + "/1/" + y
        try :
            if a[0].isdigit() :
                d = a[0]
                m = a[1]
            else :
                d = a[1]
                m = a[0]
            d = re.search(r'(\d{1,2})', d).group()
            m = myMonth(m)
            return m + "/" + d + "/" + y
        except :
            print(a)
            sys.exit()
    elif df.re2 != -1 :
        a = df.re2
        a = a.replace("-", "/")
        a = a.split("/")
        if len(a[2]) == 2 :
            a[2] = "19" + a[2]
        return a[0] + "/" + a[1] + "/" + a[2]
    else :
        a = df.re3.split("/")
        if len(a) == 1 :
            return "1/1/" + a[0]
        else :
            return a[0] + "/1/" + a[1]

In [ ]:
def date_sorter():
    re1 = "(?:\d{1,2} )?(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z.,]* (?:\d{1,2}(?:st|nd|rd|th)?[.,]? )?\d{4}"
    re2 = "\d{1,2}[/-]\d{1,2}[/-]\d{2,4}"
    re3 = "(?:\d{1,2}/)?\d{4}"

    myLines = []
    with open('dates.txt') as file:
        for i, line in enumerate(file):
            myLines.append(line.strip())
    df = pd.DataFrame(myLines, columns=["Text"])

    df11 = df.Text.str.extract(pat = "(" + re1 + ")")
    df12 = df.Text.str.extract(pat = "(" + re2 + ")")
    df13 = df.Text.str.extract(pat = "(" + re3 + ")")

    dfAll = pd.merge(df, df11.to_frame(), left_index=True, right_index=True)
    dfAll = pd.merge(dfAll, df12.to_frame(), left_index=True, right_index=True)
    dfAll = pd.merge(dfAll, df13.to_frame(), left_index=True, right_index=True)
    print(dfAll.head())
    dfAll.columns = ["Text", "re1", "re2", "re3"]
    dfAll.fillna(-1, inplace=True)

    dfAll['Date'] = dfAll.apply(turnToDate, axis = 1)
    dfAll["Date2"] = pd.to_datetime(dfAll.Date)
    dfNew = dfAll.sort_values("Date2", axis=0, ascending=True)
    dfNew.reset_index(inplace=True)
    s = dfNew["index"]
    
    return s
# date_sorter()